In [228]:
import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from isolated_ad_model.processing import PREPROCESS
from art.attacks.evasion import HopSkipJump
from art.estimators.classification import SklearnClassifier, BlackBoxClassifier
from a2pm import A2PMethod
import joblib
import logging 

logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s [%(levelname)s] %(message)s', 
    handlers=[logging.StreamHandler()]
)

logger = logging.getLogger(__name__)

In [229]:
class AnomalyModelFactory:
    def __init__(self, model_recipe = None):
        self.model_recipe = model_recipe
        self.model = None

    def save_model(self, path="model.pkl"):
        joblib.dump(self.model, path)

    def load_model(self, path="model.pkl"):
        if not os.path.exists(path):
            raise FileNotFoundError(f"No model found at {path}. Please train the model first.")
        self.model = joblib.load(path)
        return self.model

    @staticmethod
    def get_scorer(true_labels):
        def scorer(estimator, X):
            pred = estimator.predict(X)
            pred = [1 if p == -1 else 0 for p in pred]
            return f1_score(true_labels, pred)
        return scorer
        
    def _get_iso_forest(self, training_data, true_anomalies):
        random_state = 4
        parameter = {'contamination': [of for of in np.arange(0.01, 0.5, 0.02)],
                     'n_estimators': [100*(i+1) for i in range(1, 10)],
                     'max_samples': [0.005, 0.01, 0.1, 0.15, 0.2, 0.3, 0.4]}
        cv = [(slice(None), slice(None))]
        scorer = self.get_scorer(true_anomalies)
        iso = IsolationForest(random_state=random_state, bootstrap=True, warm_start=False)
        model = RandomizedSearchCV(iso, parameter, scoring=scorer, cv=cv, n_iter=50)
        md = model.fit(training_data.values)
        return md.best_estimator_
    
    def _get_random_forest(self, training_data, true_anomalies):
        random_state = 4
        parameter = {'n_estimators': [100*(i+1) for i in range(1, 12)],
                     'max_samples': [0.005, 0.01, 0.1, 0.15, 0.2, 0.3, 0.4],
                     'criterion': ["gini" , "entropy", "log_loss"]}
        random_forest = RandomForestClassifier(random_state=random_state, bootstrap=True, warm_start=False)
        model = RandomizedSearchCV(random_forest, parameter, scoring=None, cv=None, n_iter=50)
        md = model.fit(training_data.values, true_anomalies.values)
        return md.best_estimator_
        
    def build_model(self, training_data, true_anomalies, selected_model):

        model_dispatch = {
            "Random Forest": self._get_random_forest,
            "Isolation Forest": self._get_iso_forest,
        }

        if selected_model not in model_dispatch:
            raise ValueError(f"Unknown model: {selected_model}")
        
        self.model = model_dispatch[selected_model](training_data,true_anomalies)
        return self.model
        

In [230]:
class AnomalyRobustnessEvaluator:
    def __init__(self, model, training_data, true_anomalies):
        self.model = model
        self.training_data = training_data
        self.true_anomalies = true_anomalies
        
    @staticmethod
    def perform_a2pm_attack(pattern, training_data, model):
        a2pm_method = A2PMethod(pattern)
        a2pm_method.fit(training_data.values)
        
        raw_adv_training_data = a2pm_method.generate(model, training_data.values)

        return pd.DataFrame(raw_adv_training_data, columns=training_data.columns)
    
    @staticmethod
    def predict_wrapper(model, data):
        pred = model.predict(data.values)

        if isinstance(model, IsolationForest):
            return [1 if p == -1 else 0 for p in pred]
        else: 
            return pred
    
    @staticmethod
    def get_hsja_predict(model):
        def hsja_predict(data):
            pred = model.predict(data)
            if isinstance(model, IsolationForest):
                pred = [1 if p == -1 else 0 for p in pred]
            return np.eye(2)[pred] # The output needs to be one-hot
        return hsja_predict
    
    @staticmethod
    def log_evaluation(pred, true_labels, description=""):
        inliers = sum(p == 0 for p in pred)
        outliers = sum(p == 1 for p in pred)
        cm = confusion_matrix(true_labels, pred)
        
        logger.info(f"\n--- {description} ---")
        logger.info(f"Inliers: {inliers}, Outliers: {outliers}")
        logger.info("Classification Report:\n" + classification_report(true_labels, pred, zero_division=0.0))
        logger.info(f"Macro F1: {f1_score(true_labels, pred, average='macro', zero_division=0.0):.4f}")
        logger.info(f"Confusion Matrix:\n{cm}")


    def test_a2pm(self):
        if self.model is None:
            raise ValueError("Model must be trained before testing attack")

        pred = self.predict_wrapper(self.model, self.training_data)

        # TODO: Figure out patterns - how to optimize them
        # pattern = (

        #         {
        #             "type": "interval",
        #             "features": None,
        #             "ratio": 0.1,
        #             "probability": 0.6,
        #             "momentum": 0.99
        #         },
        #     )

        pattern = (

                # First pattern to be applied: Interval
                {
                    "type": "interval",
                    "features": None,
                    "integer_features": None,
                    "ratio": 0.1,
                    "max_ratio": 0.3,
                    "missing_value": 0.0,
                    "probability": 0.6,
                },

                # # Second pattern to be applied: Combination
                # {
                #     "type": "combination",
                #     "features": None,
                #     "locked_features": None,
                #     "probability": 0.4,
                # },
            )
            
        adv_training_data = self.perform_a2pm_attack(pattern, self.training_data, self.model)
        adv_pred = self.predict_wrapper(self.model, adv_training_data)

        #log results 
        self.log_evaluation(pred, self.true_anomalies, "Before A2PM Attack")
        self.log_evaluation(adv_pred, self.true_anomalies, "After A2PM Attack")

    def test_hsja(self):
        
        if self.model is None:
            raise ValueError("Model must be trained before testing attack")

        pred = self.predict_wrapper(self.model, self.training_data)

        clip_values = (self.training_data.min().min(), self.training_data.max().max()) # Extract minimum and maximum values
        input_shape = (self.training_data.shape[1],)
        hsja_predict = self.get_hsja_predict(self.model)
        classifier = BlackBoxClassifier(predict_fn=hsja_predict,input_shape=input_shape,nb_classes=2,clip_values=clip_values)
        hsja = HopSkipJump(classifier=classifier)

        np_adv_data = hsja.generate(self.training_data.values[:20], max_iter=50, max_eval=10000, init_eval=100, verbose=False)
        adv_data = pd.DataFrame(np_adv_data, columns=self.training_data.columns)

        adv_pred = self.predict_wrapper(self.model, adv_data)

        #log results 
        self.log_evaluation(pred, self.true_anomalies, "Before HSJA Attack")
        self.log_evaluation(adv_pred, self.true_anomalies[:len(adv_pred)], "After HSJA Attack")

        logger.info(adv_data.compare(self.training_data[:len(adv_pred)]))
        logger.info(f"L1 Norm {np.linalg.norm(adv_data - self.training_data[:len(adv_pred)], ord=1, axis=0)}")
        logger.info(f"L2 Norm { np.linalg.norm(adv_data - self.training_data[:len(adv_pred)], ord=2, axis=0)}")
        logger.info(f"L_inf Norm {np.linalg.norm(adv_data - self.training_data[:len(adv_pred)], ord=np.inf, axis=0)}")




In [231]:
#config
use_cached_model = True
model_path = "rf_model.pkl"
model_name = "Random Forest"

#obtain the dataset
dataset = pd.read_csv('../isolated_ad_model/ue.csv')
true_anomalies = dataset['Viavi.UE.anomalies']
ps = PREPROCESS(dataset)  # TODO: Is it possible to get rid of src/scale dependency? 
ps.process()
training_data = ps.data

#obtain the model
model_factory = AnomalyModelFactory() # TODO: Add configuration from file

if use_cached_model:
    model = model_factory.load_model(model_path) 
else:
    model = model_factory.build_model(training_data, true_anomalies, model_name)
    model_factory.save_model(model_path)

#testing enviroment
evaluator = AnomalyRobustnessEvaluator(model, training_data, true_anomalies)

In [232]:
evaluator.test_a2pm()
evaluator.test_hsja()

2025-04-07 23:23:37,294 [INFO] 
--- Before A2PM Attack ---
2025-04-07 23:23:37,295 [INFO] Inliers: 7670, Outliers: 2330
2025-04-07 23:23:37,313 [INFO] Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7432
           1       1.00      0.91      0.95      2568

    accuracy                           0.98     10000
   macro avg       0.98      0.95      0.97     10000
weighted avg       0.98      0.98      0.98     10000

2025-04-07 23:23:37,320 [INFO] Macro F1: 0.9678
2025-04-07 23:23:37,321 [INFO] Confusion Matrix:
[[7432    0]
 [ 238 2330]]
2025-04-07 23:23:37,326 [INFO] 
--- After A2PM Attack ---
2025-04-07 23:23:37,327 [INFO] Inliers: 3285, Outliers: 6715
2025-04-07 23:23:37,340 [INFO] Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.18      0.24      7432
           1       0.09      0.23      0.13      2568

    accuracy                          

In [233]:
type(model)

sklearn.ensemble._forest.RandomForestClassifier